<a href="https://colab.research.google.com/github/arina080803/itmo_ML_course_2024/blob/hw_2/itmo_course_ML_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 2

    Построить более сложную модель с подбором гиперпараметров

В рамках данного пункта необходимо использовать более сложную модель для решения задачи, оптимизировать гиперпараметры и оценить ее качество.

Критерии оценки:

    Выбрана более сложная ML-модель - 1 балл.
    Произведен подбор гиперпараметров с использованием кросс-валидации - 4 балла.
    Выбранная модель обучена с лучшими подобранными значениями гиперпараметров - 4 балла.
    Произведено измерение качества на отложенной выборке с использованием ранее выбранной метрики - 1 балл.

Предполагается, что студенты продвинутого трека среди прочего продемонстрируют использование библиотеки optuna для подбора гиперпараметров.

    Проинтерпретировать полученную модель

В рамках данного пункта необходимо проинтерпретировать модель, полученную в предыдущем пункте.

Критерии оценки:

    Получена интерпретация построенной модели, включая визуализации (коэффициенты/permutation importances/shap и тд) - 5 баллов.
    Приведено экспертное мнение о полученной интерпретации (вы, как эксперт в предметной области, можете оценить адекватность признаков и решений, принимаемых моделью, и выразить свое мнение в 1-2 предложении) - 5 баллов.


# Загрузка данных и предобработка

In [ ]:
!pip install --upgrade pandas

In [ ]:
!pip install optuna > None

In [ ]:
!pip install mljar-supervised

In [ ]:
!pip install xgboost

In [ ]:
import pandas as pd
import numpy as np


from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error

Загружаем увеличенный датасет с нормализованными данными.

In [ ]:
df = pd.read_csv('/content/sample_data/df_corr_augmented_norm_18_11.csv - Minu.csv(2).csv')
df

,Unnamed: 0,Толщина_сигмовидной_кишки_(УЗИ),Консистенция_стула,Примесь_крови_в_стуле,Кальпротектин,Мочевая_кислота,СОЭ,WBC,Индекс_UCEIS_0-
0,0,0.454545,0.5,0.333333,0.384181,0.385600,0.074074,0.220347,1.0
1,1,0.690909,1.0,1.000000,0.156497,0.573045,0.092593,0.241913,1.0
2,2,0.654545,1.0,1.000000,0.209040,0.617308,0.333333,0.247070,1.0
3,3,0.200000,0.0,0.000000,0.012723,0.819862,0.018519,0.279419,0.0
4,4,0.181818,0.0,0.000000,0.040548,0.443306,0.129630,0.000000,0.0
...,...,...,...,...,...,...,...,...,...
24010,29190,0.054545,0.0,0.000000,1.000000,0.134244,0.074074,0.079231,1.0
24011,29191,0.181818,0.0,0.000000,0.040548,0.443306,0.129630,0.000000,0.0
24012,29192,0.090909,0.0,0.000000,0.228814,0.227416,0.148148,0.152368,1.0
24013,29195,0.636364,1.0,0.333333,1.000000,0.780528,0.259259,0.110642,1.0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns='Индекс_UCEIS_0-'),
    df['Индекс_UCEIS_0-'],
    random_state=2023,
    test_size=0.3
)

y_train

2668     1.0
15606    1.0
1846     1.0
3787     0.0
10642    0.0
        ... 
6049     0.0
19127    0.0
14790    0.0
22041    0.0
21335    1.0
Name: Индекс_UCEIS_0-, Length: 16810, dtype: float64

In [ ]:
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns
)
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns
)

# Подбор гиперпараметров

Решетчатый подход выбора гиперпараметров - предполагает задание наборов значений для каждого гиперпараметра, которые затем "перебираются" систематически для нахождения наилучшей комбинации. Для каждой комбинации гиперпараметров производится обучение модели и оценка её производительности на валидационных данных.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Определение параметров и их значений для перебора
param_grid = {
    'max_depth': [1, 10, 20, 30, 40, 50],
    'min_samples_leaf': [1, 2, 4]
}

# Создание модели и настройка с использованием решетчатого поиска
rf_model = RandomForestClassifier(random_state=True)
grid_search = GridSearchCV(rf_model, param_grid, cv=2)
grid_search.fit(X_train, y_train)

# Вывод наилучших гиперпараметров и оценки
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

Best Hyperparameters: {'max_depth': 10, 'min_samples_leaf': 1}
Best Cross-Validation Score: 1.0


Другой метод — это случайный поиск гиперпараметров. Вместо того чтобы перебирать все комбинации, случайный поиск выбирает случайные наборы значений для каждого гиперпараметра. Этот метод может быть более эффективным по времени, так как он обычно требует меньше итераций, чтобы найти хорошие значения.

Случайный поиск может быть эффективнее в поиске оптимальных гиперпараметров, особенно когда ресурсы ограничены. Однако есть вероятность упустить некоторые комбинации, которые могли бы быть лучшими.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint

# Определение диапазонов значений для случайного поиска
param_dist = {
    'n_estimators': randint(1, 20), # число деревьев
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_leaf': [1, 2, 4]
}

# Создание модели и настройка с использованием случайного поиска
rf_model = RandomForestClassifier(random_state=True)
random_search = RandomizedSearchCV(rf_model, param_distributions=param_dist, n_iter=100, cv=5)
random_search.fit(X_train, y_train)

# Вывод наилучших гиперпараметров и оценки
print("Best Hyperparameters:", random_search.best_params_)
print("Best Cross-Validation Score:", random_search.best_score_)

Best Hyperparameters: {'max_depth': 50, 'min_samples_leaf': 2, 'n_estimators': 16}
Best Cross-Validation Score: 1.0


Был зафиксирован random_state=True у RandomForestClassifier, чтобы он выдавал одинаковый результат при перезапуске

Optuna — использует адаптивное методологическое пространство для эффективного поиска оптимальных значений.

In [ ]:
import optuna
from sklearn.model_selection import cross_val_score

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 1, 20)
    max_depth = trial.suggest_categorical('max_depth', [None, 10, 20, 30, 40, 50])
    min_samples_leaf = trial.suggest_categorical('min_samples_leaf', [1, 2, 4])

    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_leaf=min_samples_leaf)

    score = cross_val_score(model, X_train, y_train, cv=2).mean()
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
best_score = study.best_value

[I 2024-02-22 15:53:11,624] A new study created in memory with name: no-name-9c355aa7-b551-4efe-b2ce-a4b5fb5856a2
[I 2024-02-22 15:53:11,687] Trial 0 finished with value: 1.0 and parameters: {'n_estimators': 3, 'max_depth': 20, 'min_samples_leaf': 4}. Best is trial 0 with value: 1.0.
[I 2024-02-22 15:53:11,884] Trial 1 finished with value: 1.0 and parameters: {'n_estimators': 16, 'max_depth': 50, 'min_samples_leaf': 4}. Best is trial 0 with value: 1.0.
[I 2024-02-22 15:53:11,933] Trial 2 finished with value: 1.0 and parameters: {'n_estimators': 2, 'max_depth': 10, 'min_samples_leaf': 2}. Best is trial 0 with value: 1.0.
[I 2024-02-22 15:53:12,009] Trial 3 finished with value: 1.0 and parameters: {'n_estimators': 4, 'max_depth': None, 'min_samples_leaf': 1}. Best is trial 0 with value: 1.0.
[I 2024-02-22 15:53:12,194] Trial 4 finished with value: 1.0 and parameters: {'n_estimators': 14, 'max_depth': 10, 'min_samples_leaf': 2}. Best is trial 0 with value: 1.0.
[I 2024-02-22 15:53:12,373]

In [ ]:
X_train_short, X_valid_short, y_train_short, y_valid_short = train_test_split(X_train_scaled, y_train, test_size=0.25, random_state=2023)

In [ ]:
def run(model):
    model.fit(X_train_short, y_train_short)
    preds = model.predict(X_valid_short)
    print(model.__class__.__name__ + ' rmse  = ' + str(mean_squared_error(y_valid_short,  preds)))
    return model

In [ ]:
class StackingClassifier:
    def __init__(self, models, meta_model):
        self.models = models
        self.meta_model = meta_model
        self.n = len(models)
        self.valid = None

    def fit(self, X, y=None):
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=2023)
        self.valid = np.zeros((X_valid.shape[0], self.n))

        for i, model in enumerate(self.models):
            model.fit(X_train, y_train)
            self.valid[:, i] = model.predict(X_valid)
        self.meta_model.fit(self.valid, y_valid)
        return self

    def predict(self, X, y=None):
        X_meta = np.zeros((X.shape[0], self.n))

        for i, model in enumerate(self.models):
            X_meta[:, i] = model.predict(X)

        return self.meta_model.predict(X_meta)

In [ ]:
models = [RandomizedSearchCV(rf_model, param_grid), RandomForestClassifier(n_estimators=16, max_depth=50, min_samples_leaf=2)]
for model in models:
  run(model)

RandomizedSearchCV rmse  = 0.0
RandomForestClassifier rmse  = 0.0


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error

def run(model, X_train_short, y_train_short, X_valid_short, y_valid_short):
    model.fit(X_train_short, y_train_short)
    preds = model.predict(X_valid_short)
    print(model.__class__.__name__ + ' rmse  = ' + str(mean_squared_error(y_valid_short, preds)))
    return model

class StackingClassifier:
    def __init__(self, models, meta_model):
        self.models = models
        self.meta_model = meta_model
        self.n = len(models)
        self.valid = None

    def fit(self, X, y=None):
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=2023)
        self.valid = np.zeros((X_valid.shape[0], self.n))

        for i, model in enumerate(self.models):
            if isinstance(model, RandomizedSearchCV):
                model.fit(X_train, y_train)
                best_model = model.best_estimator_
                self.valid[:, i] = best_model.predict(X_valid)
            else:
                model.fit(X_train, y_train)
                self.valid[:, i] = model.predict(X_valid)
        self.meta_model.fit(self.valid, y_valid)
        return self

    def predict(self, X, y=None):
        X_meta = np.zeros((X.shape[0], self.n))

        for i, model in enumerate(self.models):
            if isinstance(model, RandomizedSearchCV):
                best_model = model.best_estimator_
                X_meta[:, i] = best_model.predict(X)
            else:
                X_meta[:, i] = model.predict(X)

        return self.meta_model.predict(X_meta)

# Определите параметры для поиска по сетке
param_grid = {
    'n_estimators': [16, 32, 64],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_leaf': [1, 2, 4]
}

# Создайте объект RandomizedSearchCV
rf_model = RandomForestClassifier()
random_search = RandomizedSearchCV(rf_model, param_distributions=param_grid)

# Определите список моделей
models = [random_search, RandomForestClassifier(n_estimators=16, max_depth=50, min_samples_leaf=2)]

# Создайте и обучите стекинг классификатора
stacking_classifier = StackingClassifier(models=models, meta_model=RandomForestClassifier())
stacking_classifier.fit(X_train, y_train)

# Предскажите результаты на тестовом наборе
preds = stacking_classifier.predict(X_test)
print('StackingClassifier rmse  = ' + str(mean_squared_error(y_test, preds)))

StackingClassifier rmse  = 0.0


модель переобучилась

Stacked Classification and GridSearch

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from mlxtend.classifier import StackingClassifier

# Initializing models

clf1 = KNeighborsClassifier(n_neighbors=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3],
                          meta_classifier=lr)

params = {'kneighborsclassifier__n_neighbors': [1, 5],
          'randomforestclassifier__n_estimators': [10, 50],
          'meta_classifier__C': [0.1, 10.0]}

grid = GridSearchCV(estimator=sclf,
                    param_grid=params,
                    cv=2,
                    refit=True)
grid.fit(X_train, y_train)

cv_keys = ('mean_test_score', 'std_test_score', 'params')

for r, _ in enumerate(grid.cv_results_['mean_test_score']):
    print("%0.3f +/- %0.2f %r"
          % (grid.cv_results_[cv_keys[0]][r],
             grid.cv_results_[cv_keys[1]][r] / 2.0,
             grid.cv_results_[cv_keys[2]][r]))

print('Best parameters: %s' % grid.best_params_)
print('Accuracy: %.2f' % grid.best_score_)

0.895 +/- 0.00 {'kneighborsclassifier__n_neighbors': 1, 'meta_classifier__C': 0.1, 'randomforestclassifier__n_estimators': 10}
0.895 +/- 0.00 {'kneighborsclassifier__n_neighbors': 1, 'meta_classifier__C': 0.1, 'randomforestclassifier__n_estimators': 50}
0.895 +/- 0.00 {'kneighborsclassifier__n_neighbors': 1, 'meta_classifier__C': 10.0, 'randomforestclassifier__n_estimators': 10}
0.895 +/- 0.00 {'kneighborsclassifier__n_neighbors': 1, 'meta_classifier__C': 10.0, 'randomforestclassifier__n_estimators': 50}
1.000 +/- 0.00 {'kneighborsclassifier__n_neighbors': 5, 'meta_classifier__C': 0.1, 'randomforestclassifier__n_estimators': 10}
1.000 +/- 0.00 {'kneighborsclassifier__n_neighbors': 5, 'meta_classifier__C': 0.1, 'randomforestclassifier__n_estimators': 50}
1.000 +/- 0.00 {'kneighborsclassifier__n_neighbors': 5, 'meta_classifier__C': 10.0, 'randomforestclassifier__n_estimators': 10}
1.000 +/- 0.00 {'kneighborsclassifier__n_neighbors': 5, 'meta_classifier__C': 10.0, 'randomforestclassifier_

In [ ]:
from sklearn.model_selection import GridSearchCV

# Initializing models

clf1 = KNeighborsClassifier(n_neighbors=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf1, clf1, clf2, clf3],
                          meta_classifier=lr)

params = {'kneighborsclassifier-1__n_neighbors': [1, 2],
          'kneighborsclassifier-2__n_neighbors': [1, 2],
          'randomforestclassifier__n_estimators': [10, 50],
          'meta_classifier__C': [0.1, 10.0]}

grid = GridSearchCV(estimator=sclf,
                    param_grid=params,
                    cv=2,
                    refit=True)
grid.fit(X_train, y_train)

cv_keys = ('mean_test_score', 'std_test_score', 'params')

for r, _ in enumerate(grid.cv_results_['mean_test_score']):
    print("%0.3f +/- %0.2f %r"
          % (grid.cv_results_[cv_keys[0]][r],
             grid.cv_results_[cv_keys[1]][r] / 2.0,
             grid.cv_results_[cv_keys[2]][r]))

print('Best parameters: %s' % grid.best_params_)
print('Accuracy: %.2f' % grid.best_score_)

0.542 +/- 0.00 {'kneighborsclassifier-1__n_neighbors': 1, 'kneighborsclassifier-2__n_neighbors': 1, 'meta_classifier__C': 0.1, 'randomforestclassifier__n_estimators': 10}
0.542 +/- 0.00 {'kneighborsclassifier-1__n_neighbors': 1, 'kneighborsclassifier-2__n_neighbors': 1, 'meta_classifier__C': 0.1, 'randomforestclassifier__n_estimators': 50}
0.542 +/- 0.00 {'kneighborsclassifier-1__n_neighbors': 1, 'kneighborsclassifier-2__n_neighbors': 1, 'meta_classifier__C': 10.0, 'randomforestclassifier__n_estimators': 10}
0.542 +/- 0.00 {'kneighborsclassifier-1__n_neighbors': 1, 'kneighborsclassifier-2__n_neighbors': 1, 'meta_classifier__C': 10.0, 'randomforestclassifier__n_estimators': 50}
0.763 +/- 0.00 {'kneighborsclassifier-1__n_neighbors': 1, 'kneighborsclassifier-2__n_neighbors': 2, 'meta_classifier__C': 0.1, 'randomforestclassifier__n_estimators': 10}
0.763 +/- 0.00 {'kneighborsclassifier-1__n_neighbors': 1, 'kneighborsclassifier-2__n_neighbors': 2, 'meta_classifier__C': 0.1, 'randomforestcla

# Обучение модели

In [ ]:
meta_model = KNeighborsClassifier(n_neighbors=2)

stack_model = StackingClassifier(models, meta_model)
stack_model.fit(X_train_scaled, y_train)
preds = stack_model.predict(X_test)
print(mean_squared_error(y_test, preds, squared=False))

0.6151002919752712


In [ ]:
meta_model = RandomForestClassifier(random_state=10)

stack_model = StackingClassifier(models, meta_model)
stack_model.fit(X_train_scaled, y_train)
preds = stack_model.predict(X_test)
print(mean_squared_error(y_test, preds, squared=False))

0.6151002919752712


Using Pre-fitted Classifiers

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import numpy as np

clf1 = KNeighborsClassifier(n_neighbors=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
lr = LogisticRegression()

for clf in (clf1, clf2, clf3):
    clf.fit(X_train, y_train)

In [ ]:
from mlxtend.classifier import StackingClassifier
import copy
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3],
                          meta_classifier=lr, fit_base_estimators=False)

labels = ['KNN', 'Random Forest', 'Naive Bayes', 'StackingClassifier']

sclf.fit(X_train, y_train)

print('accuracy:', np.mean(y_test == sclf.predict(X_test)))

/usr/local/lib/python3.10/dist-packages/mlxtend/classifier/stacking_classification.py:167: UserWarning: fit_base_estimators=False enforces use_clones to be `False`


accuracy: 0.8857737682165163


# Выводы

Из-за того, что выборка была увеличена не лучшим образом, результат работы некоторых моделей - переобучение (в т.ч. RMSE=0.0).
Результат модели StackingClassifier: 88%. На данный момент это лучший результат, которы удалось получить.

В дальнейшем, если продолжится работа с таким датасетом, нужно будет переписать этап увеличения кол-ва данных.

Конечно же данный результат для медицинских данных неудовлетворителен, так как от точности предсказания результата моделью напрямую зависит дальнейшее лечение пациента. Модель нужно улучшать, на данной стадии ее использовать нельзя.